## Job Generation

In [32]:
import numpy as np
import scipy

In [33]:
jobsArray = []

class Job:
    def __init__(self, arrival_time, service_time, priority):
        self.arrival_time = arrival_time
        self.service_time = service_time
        self.priority = priority
        self.pre_waiting_time = None
        self.isJobDone = False
        self.service_left = service_time
        self.service_start_time = None
        self.service_end_time = None
        self.latency_due_to_preemption = None
        

def generate_exponential_xi(n, lmbda):
    y = np.random.uniform(0, 1, n)
    x = [-np.log(1 - y[i]) / lmbda for i in range(len(y))]
    return x

def generate_si_sigma_xi(n):
    n_sorted = sorted(n)
    si_sigma_xi = []
    for i in range(len(n_sorted)):
        si_sigma_xi.append(sum(n_sorted[:i+1]))
    return si_sigma_xi

def generate_sizes(si_sigma_xi, mu):
    sizes = generate_exponential_xi(len(si_sigma_xi), mu)
    jobs = []
    for i in range(len(si_sigma_xi)):
        jobs.append(Job(si_sigma_xi[i], sizes[i], None))
    return jobs

def assign_priority(jobsArray, number_of_queues, average_service_time):
    bin_size = 2 * average_service_time / number_of_queues
    for job in jobsArray:
        priority = int(job.service_time / bin_size)
        if priority >= number_of_queues:
            priority = number_of_queues - 1
        job.priority = priority
    return jobsArray

def areJobsLeft(jobsArray):
    for job in jobsArray:
        if job.isJobDone == False:   
            return True
    return False

def getHighestPriorityJob(jobsArray,global_clock):
    index = -1
    priority = 100000
    for i, job in enumerate(jobsArray):
        if job.priority <= priority and global_clock >= job.arrival_time and job.isJobDone == False:
            priority = job.priority
            index = i 
            
    return index

def getNextJobArrival(jobsArray):
    index = -1
    for i, job in enumerate(jobsArray):
        if job.isJobDone == False:
            return i 
            
def simulate_queue(jobsArray,numJobs):
    global_clock = 0
    numJobsdone = 0
    time_slice = 2
    while numJobsdone != numJobs:
        
        highest_priority_job_index = getHighestPriorityJob(jobsArray,global_clock)
        
        if highest_priority_job_index == -1:
            index =  getNextJobArrival(jobsArray)
            global_clock = jobsArray[index].arrival_time
            highest_priority_job_index = index

        currJob = jobsArray[highest_priority_job_index]  
        if currJob.service_start_time == None:
            jobsArray[highest_priority_job_index].service_start_time = global_clock
            jobsArray[highest_priority_job_index].pre_waiting_time = global_clock - currJob.arrival_time
        
        if currJob.service_left > time_slice:
            global_clock += time_slice
            jobsArray[highest_priority_job_index].service_left -= time_slice
        else:
            global_clock += currJob.service_left
            jobsArray[highest_priority_job_index].service_left = 0
            jobsArray[highest_priority_job_index].isJobDone = True
            numJobsdone += 1
            jobsArray[highest_priority_job_index].service_end_time = global_clock 


## Queuing parameters

In [34]:
# number of jobs
n = 10
number_of_queues = 4
lmbda = 0.1
mu = 0.2

xi = generate_exponential_xi(n, lmbda)
si_sigma_xi = generate_si_sigma_xi(xi)

jobsArray = generate_sizes(si_sigma_xi, mu)

jobsArray = assign_priority(jobsArray, number_of_queues, 1/mu)
    
# Count number of jobs in each queue
count = [0] * number_of_queues
for job in jobsArray:
    count[job.priority] += 1
print("Counts per priority:", count)

# Simulate the queue with the given arrival times and sizes using sjf
simulate_queue(jobsArray,n)


Counts per priority: [6, 0, 1, 3]


In [35]:

print("Jobs:")
for job in jobsArray:
    print("Arrival Time:", job.arrival_time, "Service Time:", job.service_time, "Priority:", job.priority, "Waiting Time:", job.pre_waiting_time)


Jobs:
Arrival Time: 2.950461321107108 Service Time: 1.5214333166461629 Priority: 0 Waiting Time: 0.0
Arrival Time: 6.278466584910637 Service Time: 21.480144535086765 Priority: 3 Waiting Time: 0.0
Arrival Time: 10.026395847913316 Service Time: 0.8613032448867904 Priority: 0 Waiting Time: 0.25207073699732163
Arrival Time: 14.975535018614284 Service Time: 0.684523401355611 Priority: 0 Waiting Time: 0.1642348111831442
Arrival Time: 21.26541375941425 Service Time: 5.581103573930267 Priority: 2 Waiting Time: 0.5588794717387877
Arrival Time: 27.651956826020758 Service Time: 14.461843710741475 Priority: 3 Waiting Time: 1.7534399790625486
Arrival Time: 40.322001392541665 Service Time: 0.5403653611911098 Priority: 0 Waiting Time: 1.0833954125416412
Arrival Time: 56.7790677670928 Service Time: 18.42237654563464 Priority: 3 Waiting Time: 0.0
Arrival Time: 76.58778810197333 Service Time: 2.024698502964282 Priority: 0 Waiting Time: 0.0
Arrival Time: 107.98669631097401 Service Time: 0.401980437120160

## Average Service Time for simulation

In [36]:
# totserviceTime = 0
# for job in arrival_times_and_service_times_and_priority:
#     totserviceTime += job[1]
# avg_time = totserviceTime/n

# print(avg_time)